In [16]:
import lionagi as li

import asyncio

# Special function calls

Complicated loops and iterations are a pain for large data sets, and complex data structure 

special function call handlers are designed to reduce that pain to stay focus on the workflow

## 1. to_list & lcall (list call)

lcall is the only sync special call handler

In [17]:
# we use to_list, a helpful type converter
# it flattens a nested list, and also dropna

a = [1, None, 2, 3, [4], [5, None, 6]]
li.to_list(a, flatten=True, dropna=True)

[1, 2, 3, 4, 5, 6]

In [18]:
# create a test input list
a = range(1,6)

# create some test functions
f1 = lambda x: x**2

# the first special function calling method is called l_call (list call)
# you can operate a single function on the whole set of input list
li.lcall(a, f1)

[1, 4, 9, 16, 25]

In [19]:
def multiply_and_increment(x, factor=1, increment=0):
    print(x * factor + increment)

li.lcall(a, multiply_and_increment, factor=2, increment=3);

5
7
9
11
13


## 2. Async call handlers

LionAGI is designed to be async only, (sync functions also works)

there are many handlers for async operations

`alcall` (async list call)

In [20]:
# define an async function
async def add_1(x):
    return x+1

# let's try to run it as usual 
add_1(2)

<coroutine object add_1 at 0x1302e41c0>

In [21]:
# we need to add await in front of an async function to recieve the output, rather than a coroutine object
async def async_inverse(x):
    return 1 / x if x != 0 else None

await async_inverse(2)

0.5

In [22]:
async def async_increment(x, inverse=False):
    if inverse:
        return await async_inverse(x+1)
    return x + 1

# we can use alcall (async list call) to run an async function on a list
await li.alcall([[1, 3], [7, 9]], async_increment, flatten=True, inverse=True)

[0.5, 0.25, 0.125, 0.1]

`mcall` (mapped call) allows you to map functions and arguments to a collection of objects, and call them all in parallel.


In [24]:
f0 = lambda x: x+2
f1 = lambda x: x*2
f2 = lambda x: x**2

# apply each function to its corresponding input
await li.mcall(input_=[3,4,5], funcs=[f0, f1, f2])

[5, 8, 25]

In [28]:
# apply each function to every input, exploding the output size
await li.mcall(input_=[3,4,5,6,7], funcs=[f0, f1, f2], explode=True)

[[5, 6, 9], [6, 8, 16], [7, 10, 25], [8, 12, 36], [9, 14, 49]]

`tcall` timed call Handle both synchronous and asynchronous calls with optional delay, error handling, and execution timing.

In [29]:
async def async_function(input_):
    await asyncio.sleep(1)  # Simulate a delay
    if input_ == 'error':
        raise ValueError("Error triggered")
    return input_.upper()

# you can add a delay at the tcall
await li.tcall('hello', async_function, sleep=0.1, include_timing=True)

('HELLO', 1.1022851467132568)

In [30]:
def error_func(x):
    raise Exception("error")

await li.tcall(1, error_func, sleep=1, ignore_error=True, message="Here is an error: ")

Here is an error:  Error: error


`bcall` batch call allows the previous batch to finish before the next gets sent

In [ ]:
async def process_item(item):
    await asyncio.sleep(1)
    print(item*2)

inputs = [1, 2, 3, 4, 5, 6]
batch_size = 2

# you should see the results get printed 2 items at a time, with a 1 second delay between each batch
await li.bcall(inputs, process_item, batch_size);

2
4
6
8
10
12


In [39]:
async def process_item_with_exception(item):
    if item == 3:
        raise ValueError("Error processing item")
    await asyncio.sleep(1)
    print(item*2)
    return item * 2

# 5 and 6 didn't get processed because the function stopped after the second batch due to the exception
inputs = [1, 2, 3, 4, 5, 6]
batch_size = 2
try:
    results = await li.bcall(inputs, process_item_with_exception, batch_size)
    print(results)
except ValueError as e:
    print(f"Caught an exception: {e}")

2
4
Caught an exception: Error processing item


8
